# QM/MM in Gromos

In [1]:
# necessary imports are the Gromos_System and convenience functions emin and md
import pygromos
from pygromos.files.simulation_parameters.imd import Imd
from pygromos.files.qmmm.qmmm import QMMM
from pygromos.files.gromos_system.gromos_system import Gromos_System
from pygromos.simulations.modules.preset_simulation_modules import emin, md
from pygromos.data.simulation_parameters_templates import template_emin
from pygromos.simulations.hpc_queuing.submission_systems.local import LOCAL

# for file paths
import os

## Support for QMMM functionality in `GROMOS` input files

This notebook demonstrates support of `PyGromosTools` for QM/MM functionality.

https://github.com/rinikerlab/PyGromosTools/blob/qmmm/examples/example_gromos_qmmm.ipynb (part of the `qmmm` branch and soon to be merged to `release3`)

Author: Felix Pultar


Features include:

* QM/MM blocks in `imd` files
* QM/MM specification files
* Running QM/MM simulations

### Load an `imd` file containing a QMMM block <a class="anchor" id="imd-files"></a>
Simple demonstration of how to handle `.imd` files.

In [2]:
imd_path = os.path.abspath("example_files/QMMM_files/md.imd")
imd_file = Imd(imd_path)
imd_file.TITLE.content = "Demonstration of a Gromos imd file containing a QMMM block"
imd_file

TITLE
Demonstration of a Gromos imd file containing a QMMM block
	>>> Generated with PyGromosTools (riniker group) <<<
END
BOUNDCOND
# 	 NTB 	 NDFMIN
 	 1 	 3 	 
END
COMTRANSROT
# 	 NSCM
 	 1000 	 
END
CONSTRAINT
# 	 NTC
 	 3 	 
# 	 NTCP 	 NTCP0(1)
 	 1 	 0.000100 	 
# 	 NTCS 	 NTCS0(1)
 	 1 	 0.000100 	 
END
FORCE
# 	 BONDS 	 ANGLES 	 IMPROPER 	 DIHEDRAL 	 ELECTROSTATIC 	 VDW
 	 0 	 1 	 1 	 1 	 1 	 1 	 
# 	 NEGR 	 NRE
 	 1 	 3613 	 
END
INITIALISE
# 	 NTIVEL 	 NTISHK 	 NTINHT 	 NTINHB
 	 1 	 3 	 0 	 0 	 
# 	 NTISHI 	 NTIRTC 	 NTICOM
 	 1 	 0 	 0 	 
# 	 NTISTI
 	 0 	 
# 	 IG 	 TEMPI
 	 210185 	 298.000000 	 
END
MULTIBATH
# ALGORITHM
  0
# NBATHS
  1
# TEMP0(1 ... NBATHS) 	 TAU(1 ... NBATHS)
  298.0 	 0.1
# DOFSET
  1
# LAST(1 ... DOFSET) 	 COMBATH(1 ... DOFSET) 	 IRBATH(1 ... DOFSET)
  3613 	 1 	 1
END
NONBONDED
# 	 NLRELE
 	 1 	 
# 	 APPAK 	 RCRF 	 EPSRF 	 NSLFEXCL
 	 0.000000 	 1.400000 	 66.700000 	 1 	 
# 	 NSHAPE 	 ASHAPE 	 NA2CLC 	 TOLA2 	 EPSLS
 	 3 	 1.400000 	 2 	 1e-10 	 0.0

### Print out different sections of the QMMM block

Print out selected parameters from the `QMMM` block or also the `TITLE` block.

In [3]:
print(imd_file.QMMM.NTQMMM) # QM/MM toggled on/off
print(imd_file.QMMM.NTQMSW) # which QM/MM engine
print(imd_file.TITLE.content)

1
5
Demonstration of a Gromos imd file containing a QMMM block


### Change a block value and print again

Just change values of the `QMMM` block like with other `PyGromosTools` blocks.

In [4]:
imd_file.QMMM.NTQMSW = 4 # switch to ORCA as QM software
imd_file.QMMM

QMMM
# 	 NTQMMM 	 NTQMSW 	 RCUTQ 	 NTWQMMM 	 QMLJ 	 MMSCAL
 	 1 	 4 	 1.400000 	 0 	 0 	 -1.000000 	 
END

## Directly manipulate a QMMM specification file

The QMMM object allows to directly interact with QM/MM specification files. Future releases of `PyGromosTools` will also support generation of `QMMM` files from coordinate files (`.cnf`, `.xyz`, `.pdb`).

In [5]:
# instantiate the file object
qmmm_file_path = os.path.abspath("example_files/QMMM_files/menthol-methanol-dmf.qmmm")
qmmm_file = QMMM(qmmm_file_path)
print(qmmm_file)
# There will be warnings if more than one QM engine is selected

TITLE
menthol-methanol-dmf, qmmm specification file

	>>> Generated with PyGromosTools (riniker group) <<<
END
QMUNIT
# QLGL           QEGE           QCGC           QIGI           
0.052918 	 2625.5 	 1.0 	 0.1
END
QMZONE
# QMEN                       QMEI  QMEZ  QMEB
H                               1     1     0
C                               2     6     0
H                               3     1     0
H                               4     1     0
C                               5     6     0
H                               6     1     0
C                               7     6     0
H                               8     1     0
H                               9     1     0
H                              10     1     0
C                              11     6     0
H                              12     1     0
C                              13     6     0
H                              14     1     0
H                              15     1     0
C                              16     6   

## Print out and change some blocks in the QMMM specification file

### Title block

The `QMMM` specification file can be handled like any other `GROMOS` file.

In [6]:
print(qmmm_file.TITLE.content)
qmmm_file.TITLE.content = "Custom file header"
print(qmmm_file.TITLE.content)

['menthol-methanol-dmf, qmmm specification file\n']
Custom file header


### QMZONE block

Print out the `QMZONE` section that defines which atoms will be treated quantum-mechanically.

In [7]:
# as in other Gromos files, the first bunch of characters are ignored and used to comment, e.g. name of the atom
# second value: index of te position (starting from 1)
# third value: element number according to the PSE
# fourth value: indicate whether bond can be broken or not, default = 0
print(qmmm_file.QMZONE)

QMZONE
# QMEN                       QMEI  QMEZ  QMEB
H                               1     1     0
C                               2     6     0
H                               3     1     0
H                               4     1     0
C                               5     6     0
H                               6     1     0
C                               7     6     0
H                               8     1     0
H                               9     1     0
H                              10     1     0
C                              11     6     0
H                              12     1     0
C                              13     6     0
H                              14     1     0
H                              15     1     0
C                              16     6     0
H                              17     1     0
H                              18     1     0
C                              19     6     0
H                              20     1     0
C                          

### QMUNIT block

Print out the `QMUNIT` block that defines some unit conversions between the MD engine and the QM software.

In [8]:
# usually, these conversion factors are hard-coded in Gromos; left for historical reasons
# first value: QM length to Gromos length (e.g. Bohr to nm) 
# second value: QM energy to Gromos energy (e.g. Hartree to kJ / mol)
# third value: Gromos charge to QM charge (the same in this case)
# fourth value: QM input units to Gromos input units (e.g. Angstrom to nm)
print(qmmm_file.QMUNIT)

QMUNIT
# QLGL           QEGE           QCGC           QIGI           
0.052918 	 2625.5 	 1.0 	 0.1
END



### XTBELEMENTS block

Print and update the `XTBELEMENTS` block

In [9]:
print(qmmm_file.XTBELEMENTS)
print(qmmm_file.XTBELEMENTS.content)

XTBELEMENTS
1
6
8
END

[['1'], ['6'], ['8']]


In [10]:
# replace element numbers manually with the first ten elements of the PSE
xtbelements_new  = [[str(i) for j in range(1)] for i in range(1,11)]
print(xtbelements_new)
qmmm_file.XTBELEMENTS.content = xtbelements_new

[['1'], ['2'], ['3'], ['4'], ['5'], ['6'], ['7'], ['8'], ['9'], ['10']]


In [11]:
# show the updated section in the file
qmmm_file.XTBELEMENTS

XTBELEMENTS
1
2
3
4
5
6
7
8
9
10
END

### A helper function that returns all QM engines specified in the QM/MM specification file

There is also a sanity check in the constructor of `QMMM` to see if you did not accidentally add more than one QM engine.

In [12]:
print(qmmm_file.get_qm_engines())

['XTB']


### Store your QMMM specification file with all your other simulation files in a `Gromos_System` object

In [13]:
# that's what we want to simulate
system_name = "menthol-methanol-dmf"

# that's where we want to simulate it at
work_folder = f"example_files/{system_name}"

# create a Gromos_System object from scratch
system = Gromos_System(work_folder, system_name)

# specify prepared topology, configuration, QMMM specification file, and input file
system.top = f"example_files/QMMM_files/{system_name}-all-atom_54a7.top"
system.cnf = f"example_files/QMMM_files/{system_name}-all-atom-init_54a7.cnf"
system.qmmm = qmmm_file
system.imd = imd_file

# clean up
system.rebase_files()

# all your simulation files now live in the work folder
system


GROMOS SYSTEM: menthol-methanol-dmf
################################################################################
WORKDIR: example_files/menthol-methanol-dmf
LAST CHECKPOINT: None

GromosXX_bin: None
GromosPP_bin: None
FILES: 
	imd: /home/mlehner/PyGromosTools/docs/sphinx_project/Examples/example_files/menthol-methanol-dmf/menthol-methanol-dmf.imd
	top: /home/mlehner/PyGromosTools/docs/sphinx_project/Examples/example_files/menthol-methanol-dmf/menthol-methanol-dmf.top
	cnf: /home/mlehner/PyGromosTools/docs/sphinx_project/Examples/example_files/menthol-methanol-dmf/menthol-methanol-dmf.cnf
	qmmm: /home/mlehner/PyGromosTools/docs/sphinx_project/Examples/example_files/menthol-methanol-dmf/menthol-methanol-dmf.qmmm
FUTURE PROMISE: False
SYSTEM: 
	Solute:	['0RHW', 'TUU2', 'TUU3', 'TUU4', 'TUU5', 'TUU6', 'TUU7', 'TUU8', 'TUU9', 'TUU10', 'TUU11', 'TUU12', 'TUU13', 'TUU14', 'TUU15', 'TUU16', 'TUU17', 'TUU18', 'TUU19', 'TUU20', 'TUU21', 'TUU22', 'TUU23', 'TUU24', 'TUU25', 'TUU26', 'TUU27', 

In [14]:
# the imd file has been adapted (force groups, multibath, etc.)
system.imd

TITLE
Demonstration of a Gromos imd file containing a QMMM block
	>>> Generated with PyGromosTools (riniker group) <<<
END
BOUNDCOND
# 	 NTB 	 NDFMIN
 	 1 	 3 	 
END
COMTRANSROT
# 	 NSCM
 	 1000 	 
END
CONSTRAINT
# 	 NTC
 	 3 	 
# 	 NTCP 	 NTCP0(1)
 	 1 	 0.000100 	 
# 	 NTCS 	 NTCS0(1)
 	 1 	 0.000100 	 
END
FORCE
# 	 BONDS 	 ANGLES 	 IMPROPER 	 DIHEDRAL 	 ELECTROSTATIC 	 VDW
 	 0 	 1 	 1 	 1 	 1 	 1 	 
# 	 NEGR 	 NRE
 	 1 	 1951 	 
END
INITIALISE
# 	 NTIVEL 	 NTISHK 	 NTINHT 	 NTINHB
 	 1 	 3 	 0 	 0 	 
# 	 NTISHI 	 NTIRTC 	 NTICOM
 	 1 	 0 	 0 	 
# 	 NTISTI
 	 0 	 
# 	 IG 	 TEMPI
 	 210185 	 298.000000 	 
END
MULTIBATH
# ALGORITHM
  0
# NBATHS
  1
# TEMP0(1 ... NBATHS) 	 TAU(1 ... NBATHS)
  298.0 	 0.1
# DOFSET
  1
# LAST(1 ... DOFSET) 	 COMBATH(1 ... DOFSET) 	 IRBATH(1 ... DOFSET)
  1951 	 1 	 1
END
NONBONDED
# 	 NLRELE
 	 1 	 
# 	 APPAK 	 RCRF 	 EPSRF 	 NSLFEXCL
 	 0.000000 	 1.400000 	 66.700000 	 1 	 
# 	 NSHAPE 	 ASHAPE 	 NA2CLC 	 TOLA2 	 EPSLS
 	 3 	 1.400000 	 2 	 1e-10 	 0.0

## Run QM/MM Simulations

QM/MM calculations are possible with a special in-house build of `GROMOS`.

In [15]:
# binaries (not yet QM/MM)
gromosPP = None
gromosXX = None

# folders and title
system_name = "menthol-methanol-dmf"
work_folder =  f"example_files/{system_name}"

# files
in_cnf_path  = f"example_files/QMMM_files/menthol-methanol-dmf-all-atom-init_54a7.cnf"
in_top_path  = f"example_files/QMMM_files/menthol-methanol-dmf-all-atom_54a7.top"

# system
system = Gromos_System(
    work_folder, 
    system_name, 
    in_top_path=in_top_path, 
    in_cnf_path=in_cnf_path,
    in_gromosPP_bin_dir=gromosPP,
    in_gromosXX_bin_dir=gromosXX
)

system


GROMOS SYSTEM: menthol-methanol-dmf
################################################################################
WORKDIR: example_files/menthol-methanol-dmf
LAST CHECKPOINT: None

GromosXX_bin: None
GromosPP_bin: None
FILES: 
	imd: None
	top: example_files/QMMM_files/menthol-methanol-dmf-all-atom_54a7.top
	cnf: example_files/QMMM_files/menthol-methanol-dmf-all-atom-init_54a7.cnf
FUTURE PROMISE: False
SYSTEM: 
	Solute:	['0RHW', 'TUU2', 'TUU3', 'TUU4', 'TUU5', 'TUU6', 'TUU7', 'TUU8', 'TUU9', 'TUU10', 'TUU11', 'TUU12', 'TUU13', 'TUU14', 'TUU15', 'TUU16', 'TUU17', 'TUU18', 'TUU19', 'TUU20', 'TUU21', 'TUU22', 'TUU23', 'TUU24', 'TUU25', 'TUU26', 'TUU27', 'TUU28', 'TUU29', 'TUU30', 'TUU31', 'TUU32', 'TUU33', 'TUU34', 'TUU35', 'TUU36', 'TUU37', 'TUU38', 'TUU39', 'TUU40', 'TUU41', 'TUU42', 'TUU43', 'TUU44', 'TUU45', 'TUU46', 'TUU47', 'TUU48', 'TUU49', 'TUU50', 'TUU51', 'TUU52', 'TUU53', 'TUU54', 'TUU55', 'TUU56', 'TUU57', 'TUU58', 'TUU59', 'TUU60', 'TUU61', 'TUU62', 'TUU63', 'TUU64', 'TUU6

In [16]:
system.cnf.visualize()

AttributeError: 'Cnf' object has no attribute 'visualize'

In [17]:
# create a local submission system and specify the number of cores
submit = LOCAL(nomp=8)

# energy minimize the system
minimized_system = emin(system, submission_system=submit)

################################################################################

emin
################################################################################



/home/mlehner/PyGromosTools/pygromos/files/gromos_system/gromos_system.py:869: UserWarning: Did not change file path as its only promised None
  warnings.warn("Did not change file path as its only promised " + str(file_obj.path))
/home/mlehner/PyGromosTools/pygromos/simulations/modules/general_simulation_modules.py:114: UserWarning: Template_imd_path was used: /home/mlehner/PyGromosTools/pygromos/data/simulation_parameters_templates/emin.imd
  warnings.warn("Template_imd_path was used: " + _template_imd_path)
Traceback (most recent call last):
  File "/home/mlehner/PyGromosTools/pygromos/simulations/modules/general_simulation_modules.py", line 130, in simulation
    gromos_system.rebase_files()
  File "/home/mlehner/PyGromosTools/pygromos/files/gromos_system/gromos_system.py", line 880, in rebase_files
    self.write_files()
  File "/home/mlehner/PyGromosTools/pygromos/files/gromos_system/gromos_system.py", line 1120, in write_files
    file_obj.write(file_obj.path)
  File "/home/mlehn

Exception: Could not prepare the command block
	Could not prepare the gromos System
	Could not find directory to write to: example_files/menthol-methanol-dmf/emin/input

In [18]:
# new imd file for equilibration
in_imd_path  = f"example_files/QMMM_files/menthol-methanol-dmf-eq.imd"
minimized_system.imd = in_imd_path
minimized_system.imd.STEP.NSTLIM = 1000
minimized_system.imd.STEP.DT = 0.002 # 2.0 fs
minimized_system

NameError: name 'minimized_system' is not defined

In [19]:
# equilibrate the system
equilibrated_system = md(minimized_system, step_name="eq", submission_system=submit)
equilibrated_system

NameError: name 'minimized_system' is not defined

In [20]:
# load your favorite GCC version (required for your special GROMOS QM/MM build)
os.environ["PATH"] += "/opt/gcc-8.2.0/bin" #Change here!
os.environ["LD_LIBRARY_PATH"] = "/opt/gcc-8.2.0/lib:/home/fpultar/opt/gcc-8.2.0/lib64" #Change here!
os.environ

environ{'LESSOPEN': '| /usr/bin/lesspipe %s',
        'CONDA_PROMPT_MODIFIER': '(pygro2) ',
        'USER': 'mlehner',
        'CXXFLAGS': '-fvisibility-inlines-hidden -std=c++17 -fmessage-length=0 -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -ffunction-sections -pipe -isystem /home/mlehner/anaconda3/envs/pygro2/include',
        'OBJCOPY': '/home/mlehner/anaconda3/envs/pygro2/bin/x86_64-conda-linux-gnu-objcopy',
        'GIT_ASKPASS': '/home/mlehner/.vscode-server/bin/e18005f0f1b33c29e81d732535d8c0e47cafb0b5/extensions/git/dist/askpass.sh',
        'GCC_RANLIB': '/home/mlehner/anaconda3/envs/pygro2/bin/x86_64-conda-linux-gnu-gcc-ranlib',
        'SIZE': '/home/mlehner/anaconda3/envs/pygro2/bin/x86_64-conda-linux-gnu-size',
        'LD_LIBRARY_PATH': '/opt/gcc-8.2.0/lib:/home/fpultar/opt/gcc-8.2.0/lib64',
        'SHLVL': '2',
        'DEBUG_CXXFLAGS': '-fvisibility-inlines-hidden -std=c++17 -fmessage-length=0 -march=nocona -mtune=haswell -f

In [21]:
# new imd file for QM/MM run
in_imd_path  = f"example_files/QMMM_files/md.imd"
equilibrated_system.imd = in_imd_path
equilibrated_system.imd.STEP.NSTLIM = 100
equilibrated_system.imd.STEP.DT = 0.0005 # 0.5 fs
# qmmm specification file
equilibrated_system.qmmm = QMMM(f"example_files/QMMM_files/menthol-methanol-dmf.qmmm")

# now you want to switch to your special build of GROMOS :)
gromosXX = None # path to freshly compiled gromos "build-gcc-8.2.0-release/program"
equilibrated_system.gromosXX = gromosXX

# check if everying is correct
# note that the new .imd file and .qmmm file still live in the old location
# while the .cnf file and .top file result from a previous simulation (equilibration)
equilibrated_system

NameError: name 'equilibrated_system' is not defined

In [22]:
# print the relevant QMMM block in the new imd file
equilibrated_system.imd.QMMM

NameError: name 'equilibrated_system' is not defined

In [23]:
# go QM/MM!
production_system = md(equilibrated_system, submission_system=submit)

NameError: name 'equilibrated_system' is not defined

In [24]:
# visualize the last .cnf - you are done!
production_system.cnf.visualize()

NameError: name 'production_system' is not defined